# Walk-Forward Analysis Example

This notebook demonstrates how to run the Walk-Forward Analysis (WFA) pipeline programmatically.

In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.append(os.path.abspath('src'))

from src.utils import load_config, setup_logging
from src.data_adapter import DataAdapter
from src.structure import StructureAnalyzer
from src.features import FeatureEngineer
from src.entries import EntryGenerator
from src.backtester import Backtester
from src.wfa import WalkForwardAnalyzer

In [ ]:
# Load Config
config = load_config("config.yml")
logger = setup_logging()

In [ ]:
# 1. Load Data
adapter = DataAdapter(config)
# Assuming data exists. If not, this will fail.
try:
    df_m1 = adapter.load_data()
    print(f"Loaded {len(df_m1)} M1 bars")
except Exception as e:
    print(f"Error loading data: {e}")
    # Create dummy data for demonstration if file missing
    dates = pd.date_range('2018-01-01', '2023-01-01', freq='1min')
    df_m1 = pd.DataFrame({
        'open': 100, 'high': 101, 'low': 99, 'close': 100, 'volume': 1000
    }, index=dates)
    df_m1['close'] = df_m1['close'] + np.random.randn(len(df_m1)).cumsum() * 0.1
    df_m1['high'] = df_m1['close'] + 0.5
    df_m1['low'] = df_m1['close'] - 0.5
    df_m1['open'] = df_m1['close'].shift(1).fillna(100)

In [ ]:
# 2. Resample
df_h4 = adapter.resample_data(df_m1, config['timeframes']['h4'])
df_h1 = adapter.resample_data(df_m1, config['timeframes']['h1'])
print(f"Resampled: H4={len(df_h4)}, H1={len(df_h1)}")

In [ ]:
# 3. Feature Engineering
structure = StructureAnalyzer(df_h4, config)
features = FeatureEngineer(config)

df_h1 = features.calculate_technical_features(df_h1)
fvgs = features.detect_fvgs(df_h1)
obs = features.detect_obs(df_h1)

entry_gen = EntryGenerator(config, structure, features)
candidates = entry_gen.generate_candidates(df_h1, df_h4, fvgs, obs)

print(f"Generated {len(candidates)} candidates")

In [ ]:
# 4. Walk-Forward Analysis
if not candidates.empty:
    # Adjust config for faster demo
    config['wfa']['train_years'] = 2
    config['wfa']['test_months'] = 6
    
    wfa = WalkForwardAnalyzer(config)
    scored_candidates = wfa.run(candidates, [])
    
    # Backtest
    backtester = Backtester(config)
    history, trades = backtester.run(scored_candidates, scored_candidates['wfa_prob'])
    
    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(history['time'], history['equity'])
    plt.title("WFA Equity Curve")
    plt.show()
    
    display(trades.head())